# **Dataset Library**

In this chapter, you'll delve deeper into the capabilities of the 🤗 Datasets library. Here are some of the key questions you'll explore:

1. How to handle datasets not available on the Hugging Face Hub?
2. Techniques for slicing, dicing, and working with datasets, including using Pandas.
3. Handling large datasets that might overwhelm your system's RAM.
4. Understanding concepts like memory mapping and Apache Arrow.
5. Creating custom datasets and contributing them to the Hugging Face Hub.

Let's embark on this journey to enhance your understanding of 🤗 Datasets!

## What should I do if my dataset isn't available on the Hugging Face Hub?

You've learned how to utilize the Hugging Face Hub to fetch datasets, but there will be instances where you need to work with data stored locally on your laptop or on a remote server. In this section, we'll explore how 🤗 Datasets can be employed to load datasets that aren't accessible on the Hugging Face Hub.

### Working with local and remote datasets

🤗 Datasets simplifies the loading of local and remote datasets by providing loading scripts for various common data formats. Here are examples of loading scripts for different data formats:

- CSV & TSV: `load_dataset("csv", data_files="my_file.csv")`
- Text files: `load_dataset("text", data_files="my_file.txt")`
- JSON & JSON Lines: `load_dataset("json", data_files="my_file.jsonl")`
- Pickled DataFrames: `load_dataset("pandas", data_files="my_dataframe.pkl")`

The above illustrates that for each data format, specifying the type of loading script in the `load_dataset()` function is sufficient. Additionally, the `data_files` argument is used to provide the path to one or more files. Let's begin by loading a dataset from local files, and subsequently, we'll explore how to achieve the same with remote files.

### Loading a local dataset

For this example we’ll use the SQuAD-it dataset, which is a large-scale dataset for question answering in Italian.

The training and test splits are hosted on GitHub, so we can download them using the blow link:

https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz

https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

Once you've downloaded them, unzip the files. You can see the compressed files has SQuAD_it-train.json and SQuAD_it-test.json, and that the data is stored in the JSON format.

### Brief Note on SQuAD Dataset
The Stanford Question Answering Dataset (SQuAD) is a popular benchmark dataset in natural language processing (NLP). It's designed for evaluating machine comprehension by requiring models to read a passage of text and answer questions about it. SQuAD contains over 100,000 question-answer pairs created by crowd workers based on articles from various sources like Wikipedia.

Each question in SQuAD is associated with a specific passage or context paragraph, and the goal is for models to generate precise answers based solely on the information in that passage. This dataset has been pivotal in the development and evaluation of question-answering systems and has led to significant advancements in NLP research, especially in the field of machine reading comprehension.

Researchers use SQuAD to train, test, and benchmark different models, aiming to achieve better accuracy and understanding in reading and answering questions. It's been instrumental in the development of various models like BERT, RoBERTa, and other transformer-based architectures in NLP.

Loading a JSON file using the `load_dataset()` function involves specifying whether the dataset is in standard JSON format (resembling a nested dictionary) or JSON Lines format (JSON separated by lines). In datasets like SQuAD-it, the information is stored in a nested structure, often with text contained within a specific field. To load this dataset correctly, we'd specify the `field` argument to accommodate this structure:

In [1]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

--2023-11-29 20:20:27--  https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz [following]
--2023-11-29 20:20:28--  https://raw.githubusercontent.com/crux82/squad-it/master/SQuAD_it-train.json.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7725286 (7.4M) [application/octet-stream]
Saving to: 'SQuAD_it-train.json.gz'

SQuAD_it-train.json 100%[===================>]   7.37M  --.-KB/s    in 0.08s   

2023-11-29 20:20:28 (98.2 MB/s) - 'SQuAD_it-train.json.gz' saved [7725286/7725286]

--202

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

gzip: SQuAD_it-test.json already exists; do you wish to overwrite (y or n)? 

In [1]:
from datasets import load_dataset

squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

When loading datasets from local files, the default behavior is to generate a DatasetDict object containing at least one split, typically the train split. To verify this, you can inspect the `squad_it_dataset` object:

In [16]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

The output displays the count of rows along with the column names present in the training set. You can explore individual examples by selecting one from the train split.

In [11]:
squad_it_dataset["train"][0]

{'title': 'Terremoto del Sichuan del 2008',
 'paragraphs': [{'context': "Il terremoto del Sichuan del 2008 o il terremoto del Gran Sichuan, misurato a 8.0 Ms e 7.9 Mw, e si è verificato alle 02:28:01 PM China Standard Time all' epicentro (06:28:01 UTC) il 12 maggio nella provincia del Sichuan, ha ucciso 69.197 persone e lasciato 18.222 dispersi.",
   'qas': [{'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56cdca7862d2951400fa6826',
     'question': 'In quale anno si è verificato il terremoto nel Sichuan?'},
    {'answers': [{'answer_start': 232, 'text': '69.197'}],
     'id': '56cdca7862d2951400fa6828',
     'question': 'Quante persone sono state uccise come risultato?'},
    {'answers': [{'answer_start': 29, 'text': '2008'}],
     'id': '56d4f9902ccc5a1400d833c0',
     'question': 'Quale anno ha avuto luogo il terremoto del Sichuan?'},
    {'answers': [{'answer_start': 78, 'text': '8.0 Ms e 7.9 Mw'}],
     'id': '56d4f9902ccc5a1400d833c1',
     'question': 'Che cosa ha

That's the right approach! Having both the train and test splits within a single DatasetDict object allows for more efficient handling. Mapping each split name to its respective file using the data_files argument ensures the inclusion of both splits in a unified dataset object.

In [2]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

Having both splits within a unified object facilitates uniform preprocessing across the entire dataset, ensuring consistency in the applied transformations or cleaning methods.

Datasets simplifies the process by handling file decompression automatically. Using compressed files directly in the `data_files` argument streamlines the loading process without the need for pre-decompression steps. This means we could have skipped the process of unzipping hte file manually by pointing the `data_files` argument directly to the compressed files.

In [3]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Whether it's ZIP, TAR, or other common compression formats, 🤗 Datasets conveniently handles the decompression process upon loading, ensuring ease of use when working with compressed files directly in the `data_files` argument.

### Loading a remote dataset

When handling remote datasets, the process remains straightforward. Instead of directing the `data_files` argument to local paths, you simply assign it the URLs where the remote files are located. For instance, in the case of the SQuAD-it dataset residing on GitHub, the `data_files` parameter can point directly to the URLs hosting the SQuAD_it-*.json.gz files.

In [4]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

This retrieves the identical DatasetDict object we previously obtained, eliminating the need for manual downloading and decompression of the SQuAD_it-*.json.gz files. With this dataset at hand, let's delve into diverse data manipulation techniques!

### Toying with data subsets

In this section, we'll explore various techniques for slicing and dicing data using the 🤗 Datasets library. We'll cover operations like selecting specific columns, filtering rows based on conditions, and shuffling the dataset

Beyond Dataset.map(), 🤗 Datasets offers a range of methods to manage datasets. These functions empower you to filter rows, select columns, shuffle data, and more. Let's explore some of these to enhance our dataset manipulations.

In this instance, we'll work with the Drug Review Dataset available on the UC Irvine Machine Learning Repository. It includes patient reviews concerning different drugs, along with the treated condition and a 10-star rating reflecting patient satisfaction.

First we need to download and extract the data

https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip


In [14]:
# !curl -O "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
# !tar -xvf drugsCom_raw.zip

!wget "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
!unzip drugsCom_raw.zip

--2023-11-28 23:49:47--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'drugsCom_raw.zip.1'

drugsCom_raw.zip.1      [       <=>          ]  41.00M  31.3MB/s    in 1.3s    

2023-11-28 23:49:48 (31.3 MB/s) - 'drugsCom_raw.zip.1' saved [42989872]

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


TSV (Tab-Separated Values) functions similarly to CSV (Comma-Separated Values) but employs tabs instead of commas as the separator. To load TSV files, you can use the csv loading script and indicate the delimiter argument within the load_dataset() function, like this:

In [5]:
from datasets import load_dataset

data_files = {"train": "drugsComTrain_raw.tsv", "test": "drugsComTest_raw.tsv"}
# \t is the tab character in Python
drug_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

One effective practice during data analysis is to extract a small random sample to gain a preliminary understanding of the data structure. In 🤗 Datasets, generating a random sample involves combining the `Dataset.shuffle()` and `Dataset.select()` functions in a sequence:

In [6]:
drug_sample = drug_dataset["train"].shuffle(seed=42).select(range(1000))
# Peek at the first few examples
drug_sample[:3]

{'Unnamed: 0': [87571, 178045, 80482],
 'drugName': ['Naproxen', 'Duloxetine', 'Mobic'],
 'condition': ['Gout, Acute', 'ibromyalgia', 'Inflammatory Conditions'],
 'review': ['"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
  '"I have taken Cymbalta for about a year and a half for fibromyalgia pain. It is great\r\nas a pain reducer and an anti-depressant, however, the side effects outweighed \r\nany benefit I got from it. I had trouble with restlessness, being tired constantly,\r\ndizziness, dry mouth, numbness and tingling in my feet, and horrible sweating. I am\r\nbeing weaned off of it now. Went from 60 mg to 30mg and now to 15 mg. I will be\r\noff completely in about a week. The fibro pain is coming back, but I would rather deal with it than the side effects."',
  '"I have been taking Mobic for over a year with no side effects other than 

Sure, here is a rephrased version of the text:

To ensure consistent results, we've fixed the seed in the `Dataset.shuffle()` function. Since `Dataset.select()` requires an iterable of indices, we've passed `range(1000)` to extract the first 1,000 samples from the shuffled dataset. This initial sample reveals a few peculiarities in our dataset:

- The `Unnamed: 0` column appears to be an anonymized patient ID.
- The `condition` column contains a combination of uppercase and lowercase labels.
- The reviews vary in length and include a mix of Python line separators (`\r\n`) and HTML character codes like `&#039;`.

To confirm our hypothesis that the `Unnamed: 0` column represents anonymized patient IDs, we can use the `Dataset.unique()` function to check if the number of IDs matches the number of rows in each split.

In [7]:
for split in drug_dataset.keys():
    assert len(drug_dataset[split]) == len(drug_dataset[split].unique("Unnamed: 0"))

Our hypothesis about the `Unnamed: 0` column being anonymized patient IDs seems valid. Let's improve the dataset's clarity by renaming the `Unnamed: 0` column to something more meaningful. We can use the `DatasetDict.rename_column()` function to modify the column name in both splits simultaneously:

In [8]:
drug_dataset = drug_dataset.rename_column(
    original_column_name="Unnamed: 0", new_column_name="patient_id"
)
drug_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

Following the tokenization process discussed in Chapter 3, let's standardize all the `condition` labels using `Dataset.map()`. We can define a simple function that can be applied to all rows in each split of `drug_dataset`:

In [9]:
def lowercase_condition(example):
    return {"condition": example["condition"].lower()}


drug_dataset.map(lowercase_condition)

Map:   0%|          | 0/161297 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'lower'

Unfortunately, we've encountered an issue with our mapping function. The error indicates that some values in the `condition` column are `None`, which cannot be lowercased because they are not strings. To handle this, we can remove these rows using `Dataset.filter()`, which operates similarly to `Dataset.map()` and takes a function that receives a single sample from the dataset. Instead of defining an explicit function like:

In [10]:
def filter_nones(x):
    return x["condition"] is not None

Instead of writing an explicit function like `filter_nones` and then calling `drug_dataset.filter(filter_nones)`, we can accomplish the same task in a single line using a lambda function. In Python, lambda functions are concise functions that can be defined without explicitly naming them. They follow the general structure:

        lambda <arguments> : <expression>


The `lambda` keyword is a special term in Python that introduces anonymous functions. The `<arguments>` section is a list or set of comma-separated values that represent the function's inputs. The `<expression>` part specifies the operations to be performed. For instance, we can define a simple lambda function that squares a number using the following code:

In [11]:
lambda x : x * x

<function __main__.<lambda>(x)>

To utilize this function for a given input, it needs to be enclosed in parentheses along with the input itself:

In [12]:
(lambda x: x * x)(3)

9

In a similar manner, lambda functions can accommodate multiple arguments by separating them with commas. For instance, we can calculate the area of a triangle using the following lambda function:

In [13]:
(lambda base, height: 0.5 * base * height)(4, 8)

16.0

Lambda functions prove useful when you need to create concise, disposable functions (for more details, we recommend reading Andre Burgaud's exceptional Real Python tutorial). Within the context of 🤗 Datasets, lambda functions allow us to define straightforward map and filter operations. Let's leverage this approach to remove the `None` entries from our dataset:

In [14]:
drug_dataset = drug_dataset.filter(lambda x: x["condition"] is not None)

Once the `None` entries have been eliminated, we can proceed with normalizing our `condition` column:

In [15]:
drug_dataset = drug_dataset.map(lowercase_condition)
# Check that lowercasing worked
drug_dataset["train"]["condition"][:3]

['left ventricular dysfunction', 'adhd', 'birth control']

Excellent! After normalizing the labels, let's turn our attention to cleaning up the reviews themselves.

#### Creating new columns

When dealing with customer reviews, it's advisable to examine the number of words in each review. A review could range from a single word like "Great!" to a lengthy essay spanning thousands of words. Depending on the specific application, you may need to handle these extremes differently. To determine the number of words in each review, we'll employ a basic approach that involves splitting each text by whitespace.

Let's define a straightforward function that calculates the word count for each review:

In [16]:
def compute_review_length(example):
    return {"review_length": len(example["review"].split())}

Unlike our `lowercase_condition()` function, `compute_review_length()` yields a dictionary whose key doesn't match any of the column names in the dataset. Consequently, when `compute_review_length()` is passed to `Dataset.map()`, it will be applied to every row in the dataset, generating a new `review_length` column:

In [17]:
drug_dataset = drug_dataset.map(compute_review_length)
# Inspect the first training example
drug_dataset["train"][0]

{'patient_id': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 17}

As anticipated, our training set now includes a `review_length` column. To examine the extreme values of this new column, we can sort it using `Dataset.sort()`:

In [18]:
drug_dataset["train"].sort("review_length")[:3]

{'patient_id': [111469, 13653, 53602],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse'],
 'condition': ['hepatitis c', 'adhd', 'birth control'],
 'review': ['"Headache"', '"Great"', '"Awesome"'],
 'rating': [10.0, 10.0, 10.0],
 'date': ['February 3, 2015', 'October 20, 2009', 'November 23, 2015'],
 'usefulCount': [41, 3, 0],
 'review_length': [1, 1, 1]}

Our suspicions were confirmed. Some reviews consist of only a single word, which, while acceptable for sentiment analysis, would not provide sufficient information for the task of predicting the condition.

An alternative method for adding new columns to a dataset is the `Dataset.add_column()` function. This function enables you to supply the column as a Python list or NumPy array, making it useful in scenarios where `Dataset.map()` is not the optimal choice for your analysis.

Let's employ the `Dataset.filter()` function to eliminate reviews with less than 30 words. Similar to our approach for handling the `condition` column, we can filter out extremely short reviews by requiring that the reviews' length exceed this threshold:

In [19]:
drug_dataset = drug_dataset.filter(lambda x: x["review_length"] > 30)
print(drug_dataset.num_rows)

{'train': 138514, 'test': 46108}


As evident, this process has eliminated approximately 15% of the reviews from our original training and test sets.

The final step involves addressing the presence of HTML character codes in our reviews. Python's `html` module provides a convenient tool for unescaping these characters, as demonstrated here:

In [20]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

We'll leverage `Dataset.map()` to unescape all HTML characters within our corpus:

In [21]:
drug_dataset = drug_dataset.map(lambda x: {"review": html.unescape(x["review"])})

#### The transformative power of the `map()` method

The `Dataset.map()` method offers a `batched` argument that, when set to `True`, instructs it to send a batch of examples to the map function simultaneously. The batch size is configurable with a default value of 1,000. For instance, the previous map function that unescaped all HTML characters took a noticeable amount of time to execute (the progress bars display the elapsed time). We can expedite this process by processing multiple elements concurrently using a list comprehension.

When `batched=True`, the function receives a dictionary containing the dataset's fields, but each value is now a list of values rather than a single value. The return value of `Dataset.map()` should remain consistent: a dictionary with the fields we want to update or add to our dataset, along with a list of values. Here's an alternative method for unescaping all HTML characters using `batched=True`:


In [22]:
new_drug_dataset = drug_dataset.map(
    lambda x: {"review": [html.unescape(o) for o in x["review"]]}, batched=True
)

If you're running this code in a notebook, you'll observe that this command executes significantly faster than the previous one. This improvement is not due to the reviews already being HTML-unescaped; re-running the instruction from the previous section (without `batched=True`) will yield the same execution time as before. The reason for this performance gain is that list comprehensions are typically faster than executing the same code in a `for` loop. Additionally, accessing numerous elements simultaneously rather than one at a time contributes to the performance improvement.

Employing `Dataset.map()` with `batched=True` will prove crucial in harnessing the speed of the "fast" tokenizers, which excel at tokenizing large text collections efficiently. For instance, to tokenize all drug reviews using a fast tokenizer, we could utilize a function like this:

In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["review"], truncation=True)

As you learned in Chapter 3, the tokenizer can handle one or multiple examples, enabling us to utilize this function with or without `batched=True`. Let's seize this opportunity to compare the performance of these different approaches. In a notebook environment, you can time a single-line instruction by adding `%time` before the line of code you want to measure:

You can also time an entire cell by placing %%time at the beginning of the cell.

In [24]:
%time tokenized_dataset = drug_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

CPU times: user 26 s, sys: 234 ms, total: 26.2 s
Wall time: 10.7 s


Here are the results we obtained with and without batching, with a fast and a slow tokenizer:

![](2023-11-23-13-28-09.png)

This implies that employing a fast tokenizer with the `batched=True` option is approximately 30 times faster than its slow counterpart without batching—an incredible performance gain! This remarkable speedup stems from the use of Rust, a language that facilitates code parallelization, in the background tokenization process. Owing to these advantages, fast tokenizers are the default choice when using `AutoTokenizer` (and the reason behind their name).

Parallelization is also the driving force behind the significant speedup, nearly 6 times faster, achieved by the fast tokenizer with batching. Parallelizing a single tokenization operation is not feasible, but when tokenizing multiple texts simultaneously, the execution can be distributed across multiple processes, each handling its assigned texts. This parallel execution model enables the impressive performance gain observed.

The `Dataset.map()` function also offers parallelization capabilities. While these capabilities don't leverage Rust, they won't allow a slow tokenizer to match the performance of a fast one, but they can still provide significant benefits, particularly when using a tokenizer that lacks a fast version. To enable multiprocessing, utilize the `num_proc` argument and specify the desired number of processes in your `Dataset.map()` call:

In [25]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)


def slow_tokenize_function(examples):
    return slow_tokenizer(examples["review"], truncation=True)


tokenized_dataset = drug_dataset.map(slow_tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/138514 [00:00<?, ? examples/s]

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map (num_proc=8):   0%|          | 0/46108 [00:00<?, ? examples/s]

You can experiment with timing to determine the optimal number of processes to use; in our case, 8 seemed to yield the most significant speed gain. Here's a comparison of the performance with and without multiprocessing:

![](2023-11-23-13-25-10.png)


While multiprocessing significantly improved the performance of the slow tokenizer, it also enhanced the fast tokenizer's performance. However, this improvement is not always guaranteed. For values of `num_proc` other than 8, our tests revealed that using `batched=True` without multiprocessing was faster. As a general recommendation, we advise against using Python multiprocessing for fast tokenizers with `batched=True`.

The versatility of the `Dataset.map()` method with `batched=True` is truly remarkable. It not only simplifies processing large datasets but also enables modifying the number of elements in the dataset. This capability proves particularly valuable in scenarios where you want to generate multiple training features from a single example.

In the realm of machine learning, an "example" typically refers to the collection of "features" provided to the model for training. In certain contexts, these features correspond to the columns in a "Dataset." However, in other scenarios, such as here and in question answering, multiple features can be derived from a single example and reside within a single column.

Let’s have a look at how it works! Here we will tokenize our examples and truncate them to a maximum length of 128, but we will ask the tokenizer to return all the chunks of the texts instead of just the first one. This can be done with return_overflowing_tokens=True:

In [26]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

Let’s test this on one example before using Dataset.map() on the whole dataset:

In [27]:
result = tokenize_and_split(drug_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

[128, 49]

Consequently, our initial example in the training set was divided into two features due to exceeding the specified maximum token length during tokenization. The resulting features have lengths of 128 and 49 tokens, respectively. Let's now apply this process to all elements of the dataset!

In [28]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

ArrowInvalid: Column 8 named input_ids expected length 1000 but got length 1463

Oops! It appears that something went wrong during the tokenization process. Upon examining the error message, we discover a discrepancy in the lengths of two columns: one column has a length of 1,463, while the other has a length of 1,000. If you've reviewed the `Dataset.map()` documentation, you might recall that it specifies the number of samples passed to the mapping function. In this case, 1,000 examples were provided to the function, resulting in 1,463 new features, leading to a shape error.

The underlying issue lies in the attempt to combine two datasets of different sizes. The `drug_dataset` columns will have a specific number of examples (in our case, 1,000, as indicated by the error message), while the `tokenized_dataset` we're constructing will have more examples (1,463, as mentioned in the error message; this exceeds 1,000 because we're splitting lengthy reviews into multiple examples using `return_overflowing_tokens=True`). This mismatch is incompatible with a `Dataset`, so we either need to remove the columns from the original dataset or ensure they have the same size as in the new dataset. The former approach can be achieved using the `remove_columns` argument:

In [29]:
tokenized_dataset = drug_dataset.map(
    tokenize_and_split, batched=True, remove_columns=drug_dataset["train"].column_names
)

After removing the columns, the tokenization process proceeds without errors. We can verify that the new dataset contains significantly more elements than the original dataset by comparing their respective lengths:

In [30]:
len(tokenized_dataset["train"]), len(drug_dataset["train"])

(206772, 138514)

As we mentioned earlier, addressing the mismatched length issue can also be achieved by adjusting the size of the old columns to match that of the new ones. To accomplish this, we'll utilize the `overflow_to_sample_mapping` field that the tokenizer returns when `return_overflowing_tokens=True` is set. This field provides a mapping from a new feature index to the index of the sample it originated from. Leveraging this mapping, we can associate each key in our original dataset with a list of values of the appropriate size by replicating the values of each example as many times as it produces new features:

In [31]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["review"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

We can see it works with Dataset.map() without us needing to remove the old columns:

In [32]:
tokenized_dataset = drug_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 206772
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 68876
    })
})

This approach yields the same number of training features as the previous method, but it preserves all the original fields. If you require these fields for post-processing steps after applying your model, this approach might be preferable.

As you have witnessed, 🤗 Datasets offers a versatile toolkit for preprocessing datasets in various ways. While the processing functions provided by 🤗 Datasets will address most of your model training needs, there may be instances where you need to transition to Pandas to access more advanced features, such as `DataFrame.groupby()` or high-level visualization APIs. Fortunately, 🤗 Datasets is designed for seamless interoperability with libraries like Pandas, NumPy, PyTorch, TensorFlow, and JAX. Let's explore how this works.

#### From Datasets to DataFrames and back

To facilitate conversion between various third-party libraries, 🤗 Datasets offers the `Dataset.set_format()` function. This function exclusively alters the dataset's output format, allowing you to seamlessly switch between formats without impacting the underlying data format, which is Apache Arrow. The formatting is applied directly to the dataset. To illustrate this, let's convert our dataset to Pandas:

In [33]:
drug_dataset.set_format("pandas")

Now when we access elements of the dataset we get a pandas.DataFrame instead of a dictionary:

In [34]:
drug_dataset["train"][:3]

,patient_id,drugName,condition,review,rating,date,usefulCount,review_length
0,95260,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,141
1,92703,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,134
2,138000,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,89


Let’s create a pandas.DataFrame for the whole training set by selecting all the elements of drug_dataset["train"]:

In [35]:
train_df = drug_dataset["train"][:]

At the technical level, `Dataset.set_format()` alters the return format for the dataset's `__getitem__()` method. This implies that when attempting to create a new object like `train_df` from a `Dataset` in the `"pandas"` format, the entire dataset must be sliced to obtain a `pandas.DataFrame`. You can independently verify that the type of `drug_dataset["train"]` remains `Dataset`, regardless of the output format.

Once the dataset is converted to the "pandas" format, you can leverage the full range of Pandas functionalities. For instance, you can employ elegant chaining to calculate the class distribution within the "condition" entries:

In [36]:
frequencies = (
    train_df["condition"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "condition", "condition": "frequency"})
)
frequencies.head()

,frequency,count
0,birth control,27655
1,depression,8023
2,acne,5209
3,anxiety,4991
4,pain,4744


After completing your Pandas analysis, you can seamlessly convert the modified DataFrame back into a `Dataset` object using the `Dataset.from_pandas()` function:

In [37]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['frequency', 'count'],
    num_rows: 819
})

This concludes our exploration of the diverse preprocessing capabilities offered by 🤗 Datasets. To finalize this section, let's establish a validation set to prepare the dataset for training a classifier. Before proceeding, we'll revert the output format of `drug_dataset` from `"pandas"` to `"arrow"`:

In [38]:
drug_dataset.reset_format()

#### Creating a validation set

While we have access to a test set for evaluation, it's a prudent practice to preserve its integrity and create a distinct validation set during the development phase. Once you're satisfied with the performance of your models on the validation set, you can perform a final sanity check on the test set. This approach helps alleviate the risk of overfitting to the test set and deploying a model that performs poorly on real-world data.

🤗 Datasets offers the `Dataset.train_test_split()` function, which draws inspiration from the popular scikit-learn functionality. Let's utilize this function to divide our training set into "train" and "validation" splits (the `seed` argument is set for reproducibility):

In [39]:
drug_dataset_clean = drug_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
drug_dataset_clean["validation"] = drug_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
drug_dataset_clean["test"] = drug_dataset["test"]
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

#### Saving a dataset

While 🤗 Datasets automatically caches every downloaded dataset and the transformations applied to it, there may be instances where you need to explicitly save a dataset to disk (for example, to prevent data loss in case the cache is cleared). As illustrated in the table below, 🤗 Datasets offers three primary functions for saving datasets in various formats:

![](2023-11-23-15-31-26.png)

For example, let’s save our cleaned dataset in the Arrow format:

In [40]:
drug_dataset_clean.save_to_disk("drug-reviews")

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

This will create a directory with the following structure:

            drug-reviews/
            ├── dataset_dict.json
            ├── test
            │   ├── dataset.arrow
            │   ├── dataset_info.json
            │   └── state.json
            ├── train
            │   ├── dataset.arrow
            │   ├── dataset_info.json
            │   ├── indices.arrow
            │   └── state.json
            └── validation
                ├── dataset.arrow
                ├── dataset_info.json
                ├── indices.arrow
                └── state.json


Each split is linked to its own `dataset.arrow` table, along with some metadata stored in `dataset_info.json` and `state.json`. The Arrow format can be conceptualized as an advanced table of columns and rows, optimized for developing high-performance applications that handle and transfer large datasets.

To load a saved dataset, utilize the `load_from_disk()` function:

In [41]:
from datasets import load_from_disk

drug_dataset_reloaded = load_from_disk("drug-reviews")
drug_dataset_reloaded

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

When saving datasets in CSV or JSON formats, each split must be stored as an individual file. One approach to achieve this is to iterate through the keys and values in the `DatasetDict` object:

In [42]:
for split, dataset in drug_dataset_clean.items():
    dataset.to_json(f"drug-reviews-{split}.jsonl")

Creating json from Arrow format:   0%|          | 0/111 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/47 [00:00<?, ?ba/s]

This saves each split in JSON Lines format, where each row in the dataset is stored as a single line of JSON. Here’s what the first example looks like:

In [43]:
!head -n 1 drug-reviews-train.jsonl

{"patient_id":89879,"drugName":"Cyclosporine","condition":"keratoconjunctivitis sicca","review":"\"I have used Restasis for about a year now and have seen almost no progress.  For most of my life I've had red and bothersome eyes. After trying various eye drops, my doctor recommended Restasis.  He said it typically takes 3 to 6 months for it to really kick in but it never did kick in.  When I put the drops in it burns my eyes for the first 30 - 40 minutes.  I've talked with my doctor about this and he said it is normal but should go away after some time, but it hasn't. Every year around spring time my eyes get terrible irritated  and this year has been the same (maybe even worse than other years) even though I've been using Restasis for a year now. The only difference I notice was for the first couple weeks, but now I'm ready to move on.\"","rating":2.0,"date":"April 20, 2013","usefulCount":69,"review_length":147}


We can then use the techniques from section 2 to load the JSON files as follows:

In [44]:
data_files = {
    "train": "drug-reviews-train.jsonl",
    "validation": "drug-reviews-validation.jsonl",
    "test": "drug-reviews-test.jsonl",
}
drug_dataset_reloaded = load_dataset("json", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]


Excellent! We have successfully explored data wrangling techniques with 🤗 Datasets. 